In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import os
import re
from IPython.display import display

In [2]:
print("Krok 0: Konfiguracja środowiska")

# Tworzenie katalogów na artefakty i wyniki
ARTIFACTS_DIR = "Artifacts_Polska"
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

# Nazwy plików
LOC_FILE = 'lokalizacja.csv'
DATA_FILE_1 = 'saleflats_2024_dateAdded_polska.csv'
DATA_FILE_2 = 'saleflats_2024_newestDate_polska.csv'
OUTPUT_FILE = 'Location_Polska.csv'

# Hiperparametry
MAX_TEXT_FEATURES = 20000

Krok 0: Konfiguracja środowiska


In [3]:
print("--- START: Budowa finalnego zbioru danych ---")

# --- Krok 1: Wczytanie i przygotowanie pliku lokalizacja.csv ---
print("Wczytywanie i przygotowywanie pliku lokalizacja.csv...")
try:
    df_lokalizacja_raw = pd.read_csv(
        'lokalizacja.csv', sep=',', header=None,
        names=['Id', 'ParentId', 'Name', 'AdditionalName', 'FullName'], dtype=str
    )
    df_lokalizacja = df_lokalizacja_raw.copy()
    df_lokalizacja['Id'] = pd.to_numeric(df_lokalizacja['Id'], errors='coerce')
    df_lokalizacja['ParentId'] = df_lokalizacja['ParentId'].replace('\\N', pd.NA)
    df_lokalizacja['ParentId'] = pd.to_numeric(df_lokalizacja['ParentId'], errors='coerce')
    df_lokalizacja.dropna(subset=['Id'], inplace=True)
    df_lokalizacja['Id'] = df_lokalizacja['Id'].astype(int)
    df_lokalizacja['ParentId'] = df_lokalizacja['ParentId'].astype('Int64')
    
    location_dict = df_lokalizacja.set_index('Id').to_dict('index')
    id_to_name = df_lokalizacja.set_index('Id')['Name'].to_dict()
    id_to_parent = df_lokalizacja.set_index('Id')['ParentId'].to_dict()
    
    # Tworzenie struktur do walidacji hierarchii
    miasta_ids = df_lokalizacja[df_lokalizacja['AdditionalName'].str.contains('Miasto', na=False)]['Id'].tolist()
    dzielnice_ids = df_lokalizacja[df_lokalizacja['AdditionalName'].str.contains('Dzielnica|Osiedle', na=False)]['Id'].tolist()
    city_to_districts = df_lokalizacja[df_lokalizacja['Id'].isin(dzielnice_ids)].groupby('ParentId')['Id'].apply(list).to_dict()
    district_to_streets = df_lokalizacja[df_lokalizacja['AdditionalName'] == 'Ulica'].groupby('ParentId')['Id'].apply(list).to_dict()
    
    print(f"Plik lokalizacja.csv wczytany. Mamy {len(location_dict)} unikalnych lokalizacji.")
except FileNotFoundError:
    print("BŁĄD KRYTYCZNY: Nie znaleziono pliku lokalizacja.csv")
    exit()

# --- Krok 2: Ekstrakcja danych z plików ofert za pomocą 'usecols' ---
# UWAGA: Indeksy kolumn mogą wymagać weryfikacji. Na podstawie przykładów i błędów,
# zakładamy, że plik ma więcej kolumn, a `locationPath` jest na końcu.
# Próbujemy znaleźć ostatnią kolumnę, ale jeśli to zawiedzie, użyjemy stałej.
try:
    # Szybki odczyt pierwszego wiersza, żeby znaleźć ostatnią kolumnę
    first_row = pd.read_csv('saleflats_2024_dateAdded_polska.csv', sep=',', header=None, nrows=1, on_bad_lines='skip')
    last_col_index = first_row.shape[1] - 1
    print(f"Automatycznie wykryto, że ostatnia kolumna ma indeks: {last_col_index}")
except:
    last_col_index = 64 # Wartość awaryjna
    print(f"Nie udało się wykryć ostatniej kolumny, używam wartości awaryjnej: {last_col_index}")

COLS_TO_EXTRACT = {
    3: 'Title', 4: 'Description', 5: 'Area', 6: 'Price',
    9: 'NumberOfRooms', 14: 'Floor', 15: 'Floors', 16: 'BuiltYear',
    last_col_index: 'locationPath' # Używamy wykrytego indeksu
}

all_offers_df = []
FILES = ['saleflats_2024_dateAdded_polska.csv', 'saleflats_2024_newestDate_polska.csv']
for filepath in FILES:
    print(f"\nEkstrakcja danych z pliku: {filepath}...")
    try:
        df_chunk = pd.read_csv(
            filepath, sep=',', header=None, on_bad_lines='skip',
            usecols=list(COLS_TO_EXTRACT.keys()), low_memory=False, dtype=str, encoding='utf-8'
        )
        df_chunk.rename(columns=COLS_TO_EXTRACT, inplace=True)
        all_offers_df.append(df_chunk)
        print(f"Udało się wyekstrahować {len(df_chunk)} wierszy.")
    except Exception as e:
        print(f"Błąd podczas ekstrakcji z {filepath}: {e}")

if not all_offers_df:
    print("BŁĄD KRYTYCZNY: Nie udało się wczytać żadnych danych z plików ofert.")
    exit()

df = pd.concat(all_offers_df, ignore_index=True)
df.dropna(subset=['locationPath'], inplace=True)
df.drop_duplicates(subset=['Title', 'Description', 'locationPath'], keep='first', inplace=True)
print(f"\nPołączono i uzyskano {len(df)} unikalnych ofert.")


# --- Krok 3: Parsowanie ścieżki i tworzenie kolumn docelowych ---
print("Parsowanie ścieżek i tworzenie kolumn docelowych...")
path_cols = ['Wojewodztwo_ID', 'Powiat_ID', 'Gmina_ID', 'Miasto_ID', 'Dzielnica_ID', 'PodDzielnica_ID', 'Ulica_ID']
split_paths = df['locationPath'].str.split(',')
path_df = pd.DataFrame(split_paths.tolist(), index=df.index)
if path_df.shape[1] > 7: path_df = path_df.iloc[:, :7]
while path_df.shape[1] < 7: path_df[path_df.shape[1]] = None
path_df.columns = path_cols
path_df = path_df.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
df = pd.concat([df.reset_index(drop=True), path_df.reset_index(drop=True)], axis=1)

df['target_district_id'] = np.where(df['PodDzielnica_ID'] != 0, df['PodDzielnica_ID'], df['Dzielnica_ID'])
df['target_city_id'] = df['Miasto_ID']
df['target_street_id'] = df['Ulica_ID']

# Filtrujemy, wymagając tylko, aby miasto było określone
df = df[df['target_city_id'] != 0].copy()
df.reset_index(drop=True, inplace=True)

if len(df) == 0:
    print("BŁĄD KRYTYCZNY: Po filtrowaniu nie pozostały żadne wiersze z określonym miastem.")
else:
    print(f"Liczba ogłoszeń z określonym miastem do treningu: {len(df)}")

--- START: Budowa finalnego zbioru danych ---
Wczytywanie i przygotowywanie pliku lokalizacja.csv...
Plik lokalizacja.csv wczytany. Mamy 398832 unikalnych lokalizacji.
Automatycznie wykryto, że ostatnia kolumna ma indeks: 52

Ekstrakcja danych z pliku: saleflats_2024_dateAdded_polska.csv...
Udało się wyekstrahować 793664 wierszy.

Ekstrakcja danych z pliku: saleflats_2024_newestDate_polska.csv...
Udało się wyekstrahować 755679 wierszy.

Połączono i uzyskano 773591 unikalnych ofert.
Parsowanie ścieżek i tworzenie kolumn docelowych...
Liczba ogłoszeń z określonym miastem do treningu: 709104


In [4]:
print("\nKrok 2: Inżynieria Cech (Uproszczona i Niezawodna)")
from sklearn.impute import SimpleImputer

# --- Przetwarzanie cech numerycznych (tylko pewne kolumny) ---
# ZMIANA: Skupiamy się tylko na 'Area' i 'Price', które na pewno są poprawne.
num_features = ['Area', 'Price']
for col in num_features:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Używamy SimpleImputer, aby uzupełnić ewentualne braki
imputer = SimpleImputer(strategy='median')
df[num_features] = imputer.fit_transform(df[num_features])

scaler = StandardScaler()
X_num = scaler.fit_transform(df[num_features])
joblib.dump(scaler, os.path.join(ARTIFACTS_DIR, 'scaler.joblib'))

# --- Przetwarzanie cech tekstowych (ze zredukowaną złożonością) ---
df['text_features'] = df['Title'].fillna('') + ' ' + df['Description'].fillna('')
df['text_features'] = df['text_features'].apply(lambda x: re.sub(r'\s+', ' ', x.lower()))

MAX_TEXT_FEATURES_REDUCED = 5000
vectorizer = TfidfVectorizer(max_features=MAX_TEXT_FEATURES_REDUCED, ngram_range=(1, 1))

print("Rozpoczynam wektoryzację tekstu ze zredukowanymi parametrami...")
X_text = vectorizer.fit_transform(df['text_features'])
joblib.dump(vectorizer, os.path.join(ARTIFACTS_DIR, 'vectorizer.joblib'))

# Aktualizujemy globalną zmienną
MAX_TEXT_FEATURES = MAX_TEXT_FEATURES_REDUCED

print(f"Przygotowano cechy: {X_text.shape[1]} tekstowych (TF-IDF) i {X_num.shape[1]} numerycznych.")
print(f"Kształt macierzy X_text: {X_text.shape}")
print(f"Kształt macierzy X_num: {X_num.shape}")


Krok 2: Inżynieria Cech (Uproszczona i Niezawodna)
Rozpoczynam wektoryzację tekstu ze zredukowanymi parametrami...
Przygotowano cechy: 5000 tekstowych (TF-IDF) i 2 numerycznych.
Kształt macierzy X_text: (709104, 5000)
Kształt macierzy X_num: (709104, 2)


In [5]:
print("\nKrok 3: Przygotowanie Celów (Target) i Cech Kategorycznych (Features)")

# Mapowanie ID na indeksy 0..N-1 (tak jak wcześniej)
city_ids = sorted(df['target_city_id'].unique())
district_ids = sorted(df['target_district_id'].unique())
street_ids = sorted(df['target_street_id'].unique())

if 0 not in district_ids: district_ids.insert(0,0)
if 0 not in street_ids: street_ids.insert(0,0)

city_id_map = {id: i for i, id in enumerate(city_ids)}
district_id_map = {id: i for i, id in enumerate(district_ids)}
street_id_map = {id: i for i, id in enumerate(street_ids)}

# Zapisanie mapowań
joblib.dump(city_id_map, os.path.join(ARTIFACTS_DIR, 'city_id_map.joblib'))
joblib.dump(district_id_map, os.path.join(ARTIFACTS_DIR, 'district_id_map.joblib'))
joblib.dump(street_id_map, os.path.join(ARTIFACTS_DIR, 'street_id_map.joblib'))

inv_city_id_map = {i: id for id, i in city_id_map.items()}
inv_district_id_map = {i: id for id, i in district_id_map.items()}
inv_street_id_map = {i: id for id, i in street_id_map.items()}
joblib.dump(inv_city_id_map, os.path.join(ARTIFACTS_DIR, 'inv_city_id_map.joblib'))
joblib.dump(inv_district_id_map, os.path.join(ARTIFACTS_DIR, 'inv_district_id_map.joblib'))
joblib.dump(inv_street_id_map, os.path.join(ARTIFACTS_DIR, 'inv_street_id_map.joblib'))

# --- ZMIANA: Miasto staje się cechą wejściową, a nie celem ---
# Tworzymy wektor cech dla miasta (indeksy z mapowania)
X_city_feature = df['target_city_id'].map(city_id_map).values

# Cele (y) to teraz tylko dzielnica i ulica
y_district = df['target_district_id'].map(district_id_map).values
y_street = df['target_street_id'].map(street_id_map).values

print(f"Przygotowano zmienne wejściowe i docelowe. Liczba unikalnych klas:")
print(f"Miasta (jako cecha wejściowa): {len(city_ids)}")
print(f"Dzielnice (jako cel): {len(district_ids)}")
print(f"Ulice (jako cel): {len(street_ids)}")


Krok 3: Przygotowanie Celów (Target) i Cech Kategorycznych (Features)
Przygotowano zmienne wejściowe i docelowe. Liczba unikalnych klas:
Miasta (jako cecha wejściowa): 6933
Dzielnice (jako cel): 1940
Ulice (jako cel): 28177


In [6]:
# --- ZMIANA: Dodajemy X_city_feature do podziału ---
# Podział na zbiór treningowy i walidacyjny
X_text_train, X_text_val, X_num_train, X_num_val, X_city_train, X_city_val, y_district_train, y_district_val, y_street_train, y_street_val = train_test_split(
    X_text, X_num, X_city_feature, y_district, y_street, test_size=0.2, random_state=42
)

print(f"Rozmiar zbioru treningowego: {X_text_train.shape[0]}")
print(f"Rozmiar zbioru walidacyjnego: {X_text_val.shape[0]}")

Rozmiar zbioru treningowego: 567283
Rozmiar zbioru walidacyjnego: 141821


In [7]:
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, Embedding, Flatten

print("\nKrok 4: Budowa i Kompilacja Modelu (Zoptymalizowana Architektura)")

# --- ZMIANA: Trzy wejścia zamiast dwóch ---
input_text = Input(shape=(MAX_TEXT_FEATURES,), name='text_input', sparse=True)
input_num = Input(shape=(X_num_train.shape[1],), name='num_input')
input_city = Input(shape=(1,), name='city_input') # Nowe wejście dla ID miasta

# Ścieżka tekstowa (bez zmian)
x1 = Dense(128, activation='relu')(input_text)
x1 = Dropout(0.5)(x1)

# Ścieżka numeryczna (bez zmian)
x2 = Dense(64, activation='relu')(input_num)
x2 = Dense(32, activation='relu')(x2)

# --- ZMIANA: Nowa ścieżka dla ID miasta z warstwą Embedding ---
# Warstwa Embedding przekształci ID miasta (np. 534) na gęsty wektor (np. o długości 50)
city_embedding_layer = Embedding(input_dim=len(city_ids), output_dim=50, name='city_embedding')(input_city)
x3 = Flatten()(city_embedding_layer) # Spłaszczamy wektor do 1D

# --- ZMIANA: Połączenie trzech ścieżek ---
combined = Concatenate()([x1, x2, x3])
z = Dense(256, activation='relu')(combined)
z = Dropout(0.5)(z)

# --- ZMIANA: Dwie głowy wyjściowe zamiast trzech ---
output_district = Dense(len(district_ids), activation='softmax', name='district_output')(z)
output_street = Dense(len(street_ids), activation='softmax', name='street_output')(z)

# --- ZMIANA: Finalny model z 3 wejściami i 2 wyjściami ---
model = Model(inputs=[input_text, input_num, input_city], outputs=[output_district, output_street])

# --- ZMIANA: Kompilacja dla 2 wyjść ---
model.compile(
    optimizer='adam',
    loss={
        'district_output': 'sparse_categorical_crossentropy',
        'street_output': 'sparse_categorical_crossentropy'
    },
    metrics={
        'district_output': 'accuracy',
        'street_output': 'accuracy'
    }
)

model.summary()


Krok 4: Budowa i Kompilacja Modelu (Zoptymalizowana Architektura)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)       │ (None, 5000)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ num_input (InputLayer)        │ (None, 2)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │         640,128 │ text_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │             192 │ num_input[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_embedding (Embedding)    │ (None, 1, 50)             │         346,650 │ city_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 32)                │           2,080 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten (Flatten)             │ (None, 50)                │               0 │ city_embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 210)               │               0 │ dropout[0][0],             │
│                               │                           │                 │ dense_2[0][0],             │
│                               │                           │                 │ flatten[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 256)               │          54,016 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 256)               │               0 │ dense_3[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ district_output (Dense)       │ (None, 1940)              │         498,580 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ street_output (Dense)         │ (None, 28177)             │       7,241,489 │ dropout_1[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 8,783,135 (33.51 MB)

 Trainable params: 8,783,135 (33.51 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
print("\nKrok 5: Trening Modelu")

# --- ZMIANA: Przygotowanie list i słowników dla nowej architektury ---
# Lista wejść teraz zawiera 3 elementy
X_train_list = [X_text_train, X_num_train, X_city_train]
# Słownik wyjść (celów) zawiera 2 elementy
y_train_dict = {'district_output': y_district_train, 'street_output': y_street_train}

X_val_list = [X_text_val, X_num_val, X_city_val]
y_val_dict = {'district_output': y_district_val, 'street_output': y_street_val}

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Trening
history = model.fit(
    X_train_list,
    y_train_dict,
    validation_data=(X_val_list, y_val_dict),
    epochs=25,
    batch_size=256,
    callbacks=[early_stopping]
)

# Zapis modelu
model.save(os.path.join(ARTIFACTS_DIR, 'location_prediction_model.h5'))
print("Model został wytrenowany i zapisany.")


Krok 5: Trening Modelu
Epoch 1/25
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 150s 67ms/step - district_output_accuracy: 0.5040 - district_output_loss: 3.1271 - loss: 7.1304 - street_output_accuracy: 0.6625 - street_output_loss: 4.0033 - val_district_output_accuracy: 0.5459 - val_district_output_loss: 1.8805 - val_loss: 4.7012 - val_street_output_accuracy: 0.6714 - val_street_output_loss: 2.8207
Epoch 2/25
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 139s 63ms/step - district_output_accuracy: 0.5503 - district_output_loss: 1.8517 - loss: 4.5930 - street_output_accuracy: 0.6710 - street_output_loss: 2.7413 - val_district_output_accuracy: 0.6102 - val_district_output_loss: 1.4719 - val_loss: 3.8456 - val_street_output_accuracy: 0.6757 - val_street_output_loss: 2.3737
Epoch 3/25
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 142s 64ms/step - district_output_accuracy: 0.5939 - district_output_loss: 1.5240 - loss: 3.8559 - street_output_accuracy: 0.6732 - street_output_loss: 2.3319 - val_district_output_accuracy: 0.6400 - val_distr

Model został wytrenowany i zapisany.


In [9]:
import gc
from tqdm.notebook import tqdm

print("\nKrok 7: Predykcja i Zastosowanie Logiki Hierarchicznej (Przetwarzanie w locie)")

n_samples = df.shape[0]
chunk_size = 50000

# --- ZMIANA: Nie przewidujemy miasta, więc nie potrzebujemy listy dla niego ---
predicted_district_ids = []
predicted_street_ids = []

# --- ZMIANA: Potrzebujemy teraz cechy miasta do predykcji ---
# X_city_feature został zdefiniowany w komórce 5
X_city_input = X_city_feature

print(f"Rozpoczynam predykcję na {n_samples} próbkach w kawałkach po {chunk_size}...")

for i in tqdm(range(0, n_samples, chunk_size)):
    chunk_text = X_text[i:i + chunk_size]
    chunk_num = X_num[i:i + chunk_size]
    chunk_city_input = X_city_input[i:i + chunk_size] # Dodajemy cechę miasta
    
    # Lista wejść do modelu
    chunk_list = [chunk_text, chunk_num, chunk_city_input]
    
    # --- ZMIANA: model.predict zwraca teraz 2 wartości ---
    pred_district_chunk, pred_street_chunk = model.predict(chunk_list, batch_size=512, verbose=0)
    
    for j in range(pred_district_chunk.shape[0]):
        # --- ZMIANA: Pobieramy prawdziwe ID miasta z oryginalnego DataFrame ---
        original_df_index = i + j
        city_id = df.iloc[original_df_index]['target_city_id']
        
        # Logika dla dzielnicy (pozostaje podobna, ale oparta na znanym `city_id`)
        valid_districts_for_city = city_to_districts.get(city_id, [])
        district_pred_index = np.argmax(pred_district_chunk[j]) 
        
        if valid_districts_for_city:
            district_mask = np.zeros_like(pred_district_chunk[j])
            valid_indices = [district_id_map.get(d_id) for d_id in valid_districts_for_city if district_id_map.get(d_id) is not None]
            if valid_indices:
                district_mask[valid_indices] = 1
                if np.sum(district_mask) > 0:
                    district_pred_index = np.argmax(pred_district_chunk[j] * district_mask)
        
        district_id = inv_district_id_map.get(district_pred_index, 0)
        
        # Logika dla ulicy (pozostaje taka sama)
        valid_streets_for_district = district_to_streets.get(district_id, [])
        street_pred_index = np.argmax(pred_street_chunk[j]) 
        
        if valid_streets_for_district:
            street_mask = np.zeros_like(pred_street_chunk[j])
            valid_indices = [street_id_map.get(s_id) for s_id in valid_streets_for_district if street_id_map.get(s_id) is not None]
            if valid_indices:
                street_mask[valid_indices] = 1
                if np.sum(street_mask) > 0:
                    street_pred_index = np.argmax(pred_street_chunk[j] * street_mask)

        street_id = inv_street_id_map.get(street_pred_index, 0)
        
        predicted_district_ids.append(district_id)
        predicted_street_ids.append(street_id)
        
    del pred_district_chunk, pred_street_chunk
    gc.collect()

# --- ZMIANA: Dodajemy do ramki danych tylko przewidziane dzielnice i ulice ---
# Prawdziwe ID miasta już tam jest jako 'target_city_id'
df['predicted_district_id'] = predicted_district_ids
df['predicted_street_id'] = predicted_street_ids

print("Zakończono predykcję z logiką hierarchiczną.")


Krok 7: Predykcja i Zastosowanie Logiki Hierarchicznej (Przetwarzanie w locie)
Rozpoczynam predykcję na 709104 próbkach w kawałkach po 50000...


  0%|          | 0/15 [00:00<?, ?it/s]

Zakończono predykcję z logiką hierarchiczną.


In [10]:
print("Odtwarzam ramkę danych 'df_original'...")

# Ponownie wczytujemy dane w ten sam sposób, co w komórce 3,
# aby mieć pewność, że df_original ma identyczną strukturę jak df przed filtrowaniem.

# Szybki odczyt pierwszego wiersza, żeby znaleźć ostatnią kolumnę
try:
    first_row = pd.read_csv('saleflats_2024_dateAdded_polska.csv', sep=',', header=None, nrows=1, on_bad_lines='skip')
    last_col_index = first_row.shape[1] - 1
except:
    last_col_index = 64 # Wartość awaryjna

COLS_TO_EXTRACT = {
    3: 'Title', 4: 'Description', 5: 'Area', 6: 'Price',
    9: 'NumberOfRooms', 14: 'Floor', 15: 'Floors', 16: 'BuiltYear',
    last_col_index: 'locationPath'
}

all_offers_df_orig = []
FILES = ['saleflats_2024_dateAdded_polska.csv', 'saleflats_2024_newestDate_polska.csv']
for filepath in FILES:
    try:
        df_chunk = pd.read_csv(
            filepath, sep=',', header=None, on_bad_lines='skip',
            usecols=list(COLS_TO_EXTRACT.keys()), low_memory=False, dtype=str, encoding='utf-8'
        )
        df_chunk.rename(columns=COLS_TO_EXTRACT, inplace=True)
        all_offers_df_orig.append(df_chunk)
    except Exception:
        pass # Ignorujemy błędy, bo już raz wczytaliśmy

df_original = pd.concat(all_offers_df_orig, ignore_index=True)
df_original.drop_duplicates(subset=['Title', 'Description', 'locationPath'], keep='first', inplace=True)

print("'df_original' został pomyślnie odtworzony.")

Odtwarzam ramkę danych 'df_original'...
'df_original' został pomyślnie odtworzony.


In [11]:
print("\nKrok 7: Generowanie i Zapis Wyników")

# --- ZMIANA: Funkcja tworząca string lokalizacji ---
def create_loc_string(row):
    # Miasto bierzemy z oryginalnej, prawdziwej kolumny
    city_id = row['target_city_id']
    city = id_to_name.get(city_id, "?")
    
    # Dzielnicę i ulicę bierzemy z predykcji
    district = id_to_name.get(row['predicted_district_id'], "?")
    street = id_to_name.get(row['predicted_street_id'], "?")
    
    # Zwracamy "?" dla nieokreślonych (ID=0)
    if row['predicted_district_id'] == 0: district = '?'
    if row['predicted_street_id'] == 0: street = '?'
        
    return f"{city} > {district} > {street}"

df['Predict_Loc'] = df.apply(create_loc_string, axis=1)

# Przygotowujemy do scalenia z oryginalnym DF
df_to_merge = df[['Title', 'Description', 'Predict_Loc']].copy()
# Tworzymy unikalny klucz do merge'owania, żeby uniknąć problemów
df_to_merge['merge_key'] = df_to_merge['Title'].fillna('') + df_to_merge['Description'].fillna('')
df_original['merge_key'] = df_original['Title'].fillna('') + df_original['Description'].fillna('')

# Scalenie
df_final = pd.merge(df_original, df_to_merge[['merge_key', 'Predict_Loc']], on='merge_key', how='left')
df_final.drop(columns=['merge_key'], inplace=True, errors='ignore')
df_final.drop_duplicates(subset=['Title', 'Description'], inplace=True)
df_final['Predict_Loc'].fillna('Brak predykcji', inplace=True)

# Zapis do pliku CSV
OUTPUT_FILE = 'Location_Polska.csv'
df_final.to_csv(OUTPUT_FILE, index=False, sep=';', encoding='utf-8-sig')
print(f"Wyniki zostały zapisane do pliku: {OUTPUT_FILE}")


Krok 7: Generowanie i Zapis Wyników
Wyniki zostały zapisane do pliku: Location_Polska.csv


In [12]:
print("\nPrzykładowe 20 wierszy z wynikami:")

predicted_df = df_final[df_final['Predict_Loc'] != 'Brak predykcji']
if len(predicted_df) > 20:
    sample = predicted_df.sample(20)
else:
    sample = predicted_df.head(20)

display_cols = ['Title', 'locationPath', 'Predict_Loc']
sample_display = sample[display_cols]

def highlight_col(s):
    return ['background-color: #ffff99' if c == 'Predict_Loc' else '' for c in s.index]

styled_sample = sample_display.style.apply(highlight_col, axis=1)

display(styled_sample)


Przykładowe 20 wierszy z wynikami:


,Title,locationPath,Predict_Loc
470100,Lea/ Park Krakowski trzypokojowe sprzedamy,"14,0,0,337,3257,75966,0",Kraków > Łobzów > ?
470153,"Apartament blisko Rynku, klimatyczna kamienica","14,0,0,337,3255,63759,531341",Kraków > Stare miasto > ?
304073,Sprzedam przytulne dwupokojowe mieszkanie,"12,0,0,370,3260,94995,401393",Poznań > Raszyn > Rembertowska
470080,M4 na pierwszym piętrze z balkonem,"2,0,0,359,0,98454,0",Rybnik > ? > ?
383513,"Mieszkanie, Lubin (gm.), 48 m²","16,81,0,13136,0,0,0",Lubin > ? > ?
532472,⭐Ostatnie Piętro⭐Idealne pod Wynajem⭐U. Medyczny,"15,0,0,378,3250,97758,0",Łódź > Polesie > ?
370659,Dwa pokoje 44m2 Galeria Mokotów.,"9,0,0,368,3283,91125,0",Warszawa > Mokotów > Bokserska
236007,3 pokoje*92 r blok* rozkładowe* niski blok*zadbane,"16,0,0,366,3272,92435,0",Wrocław > Krzyki > ?
348186,REZERWACJA,"2,0,0,359,0,0,0",Rybnik > ? > ?
106843,"Dwa/trzy pokoje, balkon - Bronowice","14,0,0,337,3257,100392,304049",Kraków > Krowodrza > Osiedle bronowice nowe
